In [ ]:
#observation = 'BP'
#observation = 'DBP'
#observation = 'SBP'
#observation = 'LDL'
observation = 'HbA1c'
file_read_2 = f'read_2_{observation}.txt'
file_out = f'{observation}.txt'

In [ ]:
import pyspark
import dxpy
import dxdata
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [ ]:
import pandas as pd
import os

In [ ]:
dispensed_database_name = dxpy.find_one_data_object(classname="database", name="app*", folder="/", name_mode="glob", describe=True)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(typename="Dataset", name="app*.dataset", folder="/", name_mode="glob")["id"]

In [ ]:
spark.sql("USE " + dispensed_database_name)
spark.sql("SHOW TABLES").show(truncate=False)

In [ ]:
spark.sql("SELECT * FROM gp_clinical LIMIT 5").toPandas()

In [ ]:
result = spark.sql("SELECT * FROM gp_clinical LIMIT 0").toPandas()
result

In [ ]:
foo = pd.read_csv(file_read_2, sep='\t', header=None, dtype=object)
foo.shape[0]

In [ ]:
foo

This is for BP. Expect value1 and value2

In [ ]:
for k in foo.iloc[:,0]:
    k = k.rstrip()
    q = f"""SELECT * FROM gp_clinical
    WHERE (read_2 = '{k}' OR read_3 = '{k}')
        AND value1 <> ''
        AND value2 <> ''
        AND value1 RLIKE '^[0-9]+(\\.[0-9]+)?$'
        AND value2 RLIKE '^[0-9]+(\\.[0-9]+)?$'
    """
    df = spark.sql(q).toPandas()
    c = df.shape[0]
    print(q + f" ==> {c}")
    if c > 0:
        result = pd.concat([result, df])

This is for DBP, SBP, LDL, HbA1c. Expect value1 or value2

In [ ]:
for k in foo.iloc[:,0]:
    k = k.rstrip()
    q = f"""SELECT * FROM gp_clinical
    WHERE (read_2 = '{k}' OR read_3 = '{k}')
        AND (value1 <> '' OR value2 <> '')
        AND (value1 RLIKE '^[0-9]+(\\.[0-9]+)?$' OR value2 RLIKE '^[0-9]+(\\.[0-9]+)?$')
    """
    df = spark.sql(q).toPandas()
    c = df.shape[0]
    print(q + f" ==> {c}")
    if c > 0:
        result = pd.concat([result, df])

In [ ]:
print(result.shape[0])
result = result.drop_duplicates()
print(result.shape[0])

In [ ]:
all_columns = result.columns.to_list()
result.sort_values(by=all_columns, inplace=True)

In [ ]:
result.head(5)

In [ ]:
result.to_csv(file_out, sep='\t', index=False)

In [ ]:
os.system(f'dx upload {file_out}')